In [40]:
import pandas as pd
import datetime

In [41]:
# Define function- Rename columns

def covid_rename_columns(input_data):
    output_data=input_data.rename(
                                columns = {'Province/State':'subregion'
                                         ,'Country/Region':'country'
                                         ,'Lat':'lat'
                                         ,'Long':'long'
                                         }
    )
    return(output_data)
    

In [42]:
# Define Function- Fill missing

def covid_fill_missing(input_data):
    output_data=input_data.fillna(value={'subregion':''})  #This will fill in NaN values with an empty string.
    return(output_data)

In [43]:
# Define function- Melt data (melt is used when we need to transform data from wide form to long form)

def covid_melt_data(input_data , value_var_name):
    output_data=input_data.melt(id_vars=['country','subregion','lat','long']
                               ,var_name='date_raw'
                               ,value_name=value_var_name
                               )
    return(output_data)

In [44]:
# Define function- Convert Dates

def covid_convert_dates(input_data):
    output_data=input_data.assign(
    date=pd.to_datetime(input_data.date_raw, format="%m/%d/%y"))
    output_data.drop(columns=['date_raw'], inplace=True)
    return(output_data)

In [45]:
# Define function- Rearrange Data

def covid_rearrange_data(input_data, value_var_name):
    output_data=(input_data
                 .filter(['country','subregion','date','lat','long',value_var_name])
                 .sort_values(['country','subregion','date','lat','long'])
                 .reset_index(drop=True)
                )
    return(output_data)

In [46]:
# Define function- Get and Wrangle data

def covid_get_data(input_url, value_var_name):
    covid_data_inprocess=pd.read_csv(input_url)
    covid_data_inprocess=covid_rename_columns(covid_data_inprocess)
    covid_data_inprocess=covid_fill_missing(covid_data_inprocess)
    covid_data_inprocess=covid_melt_data(covid_data_inprocess,value_var_name)
    covid_data_inprocess=covid_convert_dates(covid_data_inprocess)
    covid_data_inprocess=covid_rearrange_data(covid_data_inprocess, value_var_name)
    return(covid_data_inprocess)

In [47]:
# Get Data

url_confirmed='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_death='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

covid_confirmed=covid_get_data(url_confirmed, 'confirmed')
covid_death=covid_get_data(url_death, 'dead')
covid_recovered=covid_get_data(url_recovered, 'recovered')

In [48]:
# print records
print(covid_confirmed)

           country subregion       date   lat  long  confirmed
0      Afghanistan           2020-01-22  33.0  65.0          0
1      Afghanistan           2020-01-23  33.0  65.0          0
2      Afghanistan           2020-01-24  33.0  65.0          0
3      Afghanistan           2020-01-25  33.0  65.0          0
4      Afghanistan           2020-01-26  33.0  65.0          0
...            ...       ...        ...   ...   ...        ...
20772     Zimbabwe           2020-04-05 -20.0  30.0          9
20773     Zimbabwe           2020-04-06 -20.0  30.0         10
20774     Zimbabwe           2020-04-07 -20.0  30.0         11
20775     Zimbabwe           2020-04-08 -20.0  30.0         11
20776     Zimbabwe           2020-04-09 -20.0  30.0         11

[20777 rows x 6 columns]


The other 2 datasets contain data with a similar structure, except covid_deaths contains deaths, and covid_recovered contains recovered cases.

In [49]:
print(covid_death)

           country subregion       date   lat  long  dead
0      Afghanistan           2020-01-22  33.0  65.0     0
1      Afghanistan           2020-01-23  33.0  65.0     0
2      Afghanistan           2020-01-24  33.0  65.0     0
3      Afghanistan           2020-01-25  33.0  65.0     0
4      Afghanistan           2020-01-26  33.0  65.0     0
...            ...       ...        ...   ...   ...   ...
20772     Zimbabwe           2020-04-05 -20.0  30.0     1
20773     Zimbabwe           2020-04-06 -20.0  30.0     1
20774     Zimbabwe           2020-04-07 -20.0  30.0     2
20775     Zimbabwe           2020-04-08 -20.0  30.0     3
20776     Zimbabwe           2020-04-09 -20.0  30.0     3

[20777 rows x 6 columns]


In [50]:
print(covid_recovered)

           country subregion       date   lat  long  recovered
0      Afghanistan           2020-01-22  33.0  65.0          0
1      Afghanistan           2020-01-23  33.0  65.0          0
2      Afghanistan           2020-01-24  33.0  65.0          0
3      Afghanistan           2020-01-25  33.0  65.0          0
4      Afghanistan           2020-01-26  33.0  65.0          0
...            ...       ...        ...   ...   ...        ...
19666     Zimbabwe           2020-04-05 -20.0  30.0          0
19667     Zimbabwe           2020-04-06 -20.0  30.0          0
19668     Zimbabwe           2020-04-07 -20.0  30.0          0
19669     Zimbabwe           2020-04-08 -20.0  30.0          0
19670     Zimbabwe           2020-04-09 -20.0  30.0          0

[19671 rows x 6 columns]


In [51]:
# Get record counts

print(len(covid_confirmed))
print(len(covid_death))
print(len(covid_recovered))

20777
20777
19671


## MERGE DATA

In [52]:
# Drop unnecessary columns
covid_death.drop(columns=['lat','long'], inplace= True)
covid_recovered.drop(columns=['lat','long'], inplace= True)

In [53]:
#Merge Data
covid_data=(covid_confirmed
           .merge(covid_death, on=['country','subregion','date'], how='left')
           .merge(covid_recovered, on=['country','subregion','date'], how='left'))

In [54]:
# print records
print(covid_data)

           country subregion       date   lat  long  confirmed  dead  \
0      Afghanistan           2020-01-22  33.0  65.0          0     0   
1      Afghanistan           2020-01-23  33.0  65.0          0     0   
2      Afghanistan           2020-01-24  33.0  65.0          0     0   
3      Afghanistan           2020-01-25  33.0  65.0          0     0   
4      Afghanistan           2020-01-26  33.0  65.0          0     0   
...            ...       ...        ...   ...   ...        ...   ...   
20772     Zimbabwe           2020-04-05 -20.0  30.0          9     1   
20773     Zimbabwe           2020-04-06 -20.0  30.0         10     1   
20774     Zimbabwe           2020-04-07 -20.0  30.0         11     2   
20775     Zimbabwe           2020-04-08 -20.0  30.0         11     3   
20776     Zimbabwe           2020-04-09 -20.0  30.0         11     3   

       recovered  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  
...          ...  
20

In [55]:
new_cases_all=(covid_data
              .sort_values(by=['country','subregion','date'])
              .filter(['country','subregion','date','confirmed'])
              .groupby(['country','subregion'])
              .confirmed
              .diff())

In [56]:
covid_data=covid_data.assign(new_cases=new_cases_all)

In [57]:
print(covid_data.tail())

        country subregion       date   lat  long  confirmed  dead  recovered  \
20772  Zimbabwe           2020-04-05 -20.0  30.0          9     1        0.0   
20773  Zimbabwe           2020-04-06 -20.0  30.0         10     1        0.0   
20774  Zimbabwe           2020-04-07 -20.0  30.0         11     2        0.0   
20775  Zimbabwe           2020-04-08 -20.0  30.0         11     3        0.0   
20776  Zimbabwe           2020-04-09 -20.0  30.0         11     3        0.0   

       new_cases  
20772        0.0  
20773        1.0  
20774        1.0  
20775        0.0  
20776        0.0  


In [58]:
covid_data.query('country == "India"')

,country,subregion,date,lat,long,confirmed,dead,recovered,new_cases
11297,India,,2020-01-22,21.0,78.0,0,0,0.0,NaN
11298,India,,2020-01-23,21.0,78.0,0,0,0.0,0.0
11299,India,,2020-01-24,21.0,78.0,0,0,0.0,0.0
11300,India,,2020-01-25,21.0,78.0,0,0,0.0,0.0
11301,India,,2020-01-26,21.0,78.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
11371,India,,2020-04-05,21.0,78.0,3588,99,229.0,506.0
11372,India,,2020-04-06,21.0,78.0,4778,136,375.0,1190.0
11373,India,,2020-04-07,21.0,78.0,5311,150,421.0,533.0
11374,India,,2020-04-08,21.0,78.0,5916,178,506.0,605.0


In [60]:
print(covid_data.loc[(covid_data['country']=='India') & (covid_data['confirmed']>0), :].head(10))

      country subregion       date   lat  long  confirmed  dead  recovered  \
11305   India           2020-01-30  21.0  78.0          1     0        0.0   
11306   India           2020-01-31  21.0  78.0          1     0        0.0   
11307   India           2020-02-01  21.0  78.0          1     0        0.0   
11308   India           2020-02-02  21.0  78.0          2     0        0.0   
11309   India           2020-02-03  21.0  78.0          3     0        0.0   
11310   India           2020-02-04  21.0  78.0          3     0        0.0   
11311   India           2020-02-05  21.0  78.0          3     0        0.0   
11312   India           2020-02-06  21.0  78.0          3     0        0.0   
11313   India           2020-02-07  21.0  78.0          3     0        0.0   
11314   India           2020-02-08  21.0  78.0          3     0        0.0   

       new_cases  
11305        1.0  
11306        0.0  
11307        0.0  
11308        1.0  
11309        1.0  
11310        0.0  
11311   

India's first confirmed case of coronavirus came on 30th January